In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
pip install rarfile

In [7]:
import rarfile
import os

rar_path = "/content/drive/MyDrive/kits23/pretrained_models.rar"
extract_path = "/content/extracted_models"
os.makedirs(extract_path, exist_ok=True)

# Make sure unrar is installed
# !apt-get install unrar -y   # Uncomment if running on Colab/Linux

rf = rarfile.RarFile(rar_path)
rf.extractall(extract_path)
rf.close()

print("Extracted RAR to:", extract_path)


Extracted RAR to: /content/extracted_models
